# The Iris Project
___

Feito por:

- Rafael Almada

- Rafael Rosensvaig

Projeto de Visão Computacional $\sim$
Engenharia da Computação $\sim$
Insper

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
try:
    import G6_iris_recognition
except:
    !pip install G6-iris-recognition
    import G6_iris_recognition

In [ ]:
img = cv2.imread("images/0042/0042_003.bmp")
plt.imshow(img)
plt.show()

In [ ]:
def filtered_files(nod, nof):
  '''
  nod : Number of Directories
  nof : Number of Files
  '''
  for i in range(nod):
    dir = "000"+str(i)
    for j in range(nof):
        file = dir + "_00" + str(j) + ".bmp"
        path_to_img = "images/" + dir + "/" + file
        print("reading: " + path_to_img)
        img = cv2.imread(path_to_img)
        
#         new_path = "images/" + dir + "/filtered_" + file
#         cv.imwrite(new_path, imgeq)
#         print("new file saved: " + new_path + " -- OK")

In [ ]:
bw_img = cv2.cvtColor(cv2.imread("images/0042/0042_003.bmp"), cv2.COLOR_BGR2GRAY)
kernel = np.array([[1, 2, 1],
                   [2, 4, 2],
                   [1, 2, 1]], dtype="int")/16.0
# imgf = cv2.filter2D(bw_img, -1, kernel)
# imgf = cv2.GaussianBlur(bw_img,(5,5),0)
imgf = bw_img.copy()
for i in range(5):
    imgf = cv2.medianBlur(imgf,5)
    plt.imshow(imgf, cmap='gray')
    plt.show()
# plt.imshow(imgf)

In [ ]:
img = cv2.cvtColor(cv2.imread("images/0042/0042_003.bmp"), cv2.COLOR_BGR2GRAY)
print("Antes de equalizar o hist:")
plt.imshow(img, cmap='gray')
plt.show()
hist,bins = np.histogram(img.flatten(),256,[0,256])
plt.hist(img.ravel(), bins=256, range=[0,255])
plt.show()
print("Depois de equalizar o histograma:")
imgeq = cv2.equalizeHist(img)
plt.imshow(imgeq, cmap='gray')
plt.show()
hist, bins = np.histogram(imgeq.flatten(), 256, [0, 256])
plt.hist(imgeq.ravel(), bins=256, range=[0,255])
plt.show()

In [ ]:
def aaa(img, T, v):
    if T is None:
        T = np.median(img)
    new = img.copy()
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            if (img[row, col] > T):
                continue
            else:
                new[row, col] *= 1 + v
    return new

In [ ]:
imgf = imgeq.copy()
sharpen = np.array([[ 0, -1, 0],
                    [-1, 5, -1],
                    [ 0, -1, 0]])
imgf = cv2.filter2D(imgf, -1, sharpen)
plt.imshow(imgf, cmap='gray')

In [ ]:
new = aaa(imgf, np.median(imgf), 0.5)
plt.imshow(new, cmap = 'gray')

In [ ]:
def contrast(img, v, T = None):
    '''
    Função para aumentar o contraste de uma imagem
    img => Imagem em Gray Scale
    v => Valor do contraste em porcentagem
    T => Threshold (padrão = np.median(img))
    '''
    if T is None:
        T = np.median(img)
        
    out = img.copy()
    a = 1 + v
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            if (img[row, col] > T):
                out[row, col] *= a
            else:
                out[row, col] /= a
    return out

In [ ]:
imgc = cv2.cvtColor(cv2.filter2D(new, -1, sharpen), cv2.COLOR_GRAY2BGR)
plt.imshow(imgc, cmap='gray')

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
class ImageFilter(object):
    '''
    Classe automatizada de filtragem de imagens.
    Feito por: Rafael Almada e Rafael Vieira Rosenzvaig
    Insper - Engenharia da Computação - Visão Computacional 2020.2
    '''
    
    def __init__(self, verbose = False, single = False):
        '''
        Vebose : Se verdadeiro, usuário irá receber feedback enquanto a aplicação roda
        Single : Funcionalidade para testes únicos de filtro
        '''
        import time
        self.t0 = time.time()
        self.verbose = verbose
        if (single == False):
            self.start()
        else:
            c = 0
            wanted_filters = []
            while (True):
                print("Insira o nome do filtro " + str(c) + ":")
                f = str(input("> "))
                print(f)
                if (f != "end"):
                    wanted_filters.append(f)
                    c += 1
                else:
                    break
                    
            print("wanted = " + str(wanted_filters))
            print("Insira agora o path para a imagem do teste: ")
            the_img = str(input("> "))
            img = cv2.cvtColor(cv2.imread(the_img), cv2.COLOR_BGR2GRAY)
            self.img = self.single_shot(wanted_filters, img)
    
    def single_shot(self, wanted_filters, img):
        '''
        Função criada para o teste de filtros em uma imagem só.
        Usuário deverá passar uma lista "wanted_filters" com o nome dos filtros desejados e já implementados
        Além disso, é necessário passar as imagens já em preto e branco como argumento
        '''
        out = img.copy()
        for f in wanted_filters:
            if (self.verbose == True):
                print("Aplicando Filtro " + f + "...")
                try:
                    filter_method = getattr(self, f)
                    out = filter_method()
                except AttributeError:
                    raise NotImplementedError("Class `{}` does not implement `{}`".format(self.__class__.__name__, f))
        self.t1 = time.time()
        elapsed = self.t1 - self.t0
        if (verbose == True):
            print("Tempo utilizado para aplicar os filtros: " + str(elapsed))
        return out
            
    
    def start(self):
        nod = 60
        nof = 20
        print("====Iniciando tratamento das imagens====\n")
        self.organiza_arquivos(nod, nof)
        
    def set_image(self, path_to_img):
        '''
        Função que coloca uma imagem como a imagem central do objeto da classe
        path_to_img : Literalmente o path para encontrar a imagem
        '''
        self.image = cv2.cvtColor(cv2.imread(path_to_img), cv2.COLOR_BGR2GRAY)
        my_output = self.image.copy()
        self.out = my_output
        
    def eq_histogram(self):
        return cv2.equalizeHist(self.image)
    
    def sharpening(self, imgf):
        sharpen_kernel = np.array([[ 0, -1,  0],
                                   [-1,  5, -1],
                                   [ 0, -1,  0]])
        imgf = cv2.filter2D(imgf, -1, sharpen_kernel)
        return imgf
    
    def clareador(self, img, T = None, v = 0.5):
        '''
        Essa função tem por objtivo clarear parte da imagem que está mais escura, sem clarear a imagem por inteiro
        T => Threshold
        v => % de aumento na luminosidade do pixelb
        '''
        if T is None:
            T = np.median(img)
        new = img.copy()
        for row in range(img.shape[0]):
            for col in range(img.shape[1]):
                if (img[row, col] > T):
                    continue
                else:
                    new[row, col] *= 1 + v
        return new
    
    def contrast(img, v, T = None):
        '''
        Função para aumentar o contraste de uma imagem
        img => Imagem em Gray Scale
        v => Valor do contraste em porcentagem
        T => Threshold (padrão = np.median(img))
        '''
        if T is None:
            T = np.median(img)

        out = img.copy()
        a = 1 + v
        for row in range(img.shape[0]):
            for col in range(img.shape[1]):
                if (img[row, col] > T):
                    out[row, col] *= a
                else:
                    out[row, col] /= a
        return out
    
    def aplica_filtros(self):
        '''
        Rotina de uso dos filtros na imagem
        '''
        imgeq = self.eq_histogram()
        imgf = self.sharpening(imgeq)
        new = self.clareador(imgf)
        self.out = self.sharpening(new)
        
    def salva_imagem(self, new_path):
        cv2.imwrite(new_path, self.out)
        if (self.verbose):
            print("new file saved: " + new_path + " -- OK")

    def organiza_arquivos(self, nod, nof):
        '''
        nod : Number of Directories
        nof : Number of Files
        '''
        # Primeiro iremos atrás de imagem por imagem do diretório "Images/" para filtrar
        for i in range(nod):
            if (i < 10):
                dir = "000" + str(i)
            else:
                dir = "00" + str(i)
            for j in range(nof):
                if (j < 10):
                    file = dir + "_00" + str(j) + ".bmp"
                else:
                    file = dir + "_0" + str(j) + ".bmp"
                path_to_img = "images/" + dir + "/" + file
                if (self.verbose):
                    print("reading: " + path_to_img)
                
                # Coloca a nova imagem em questão
                self.set_image(path_to_img)
                
                # Aplica os filtros na ordem dada à imagem
                self.aplica_filtros()
                
                # Salvando imagem com o filtro
                new_path = "images/" + dir + "/filtered_" + file
                self.salva_imagem(new_path)
        self.t1 = time.time()
        elapsed = self.t1 - self.t0
        
        print("\n====Fim da filtragem!====")
        print("Tempo decorrido: " + str(elapsed))

In [6]:
# O uso da classe é o seguinte:

# Primeiro é necessário se ter a base com todas as imagens já em "./images/"
# Depois criamos um objeto usando a classe
## Caso o usuário deseje, pode passar o argumento verbose como True para ter uma resposta da aplicação enquanto ela roda, mas é possível não passar nada também!
## Ex: f_obj = ImageFilter()

#f_obj = ImageFilter(verbose = True)

# Depois esperamos a classe terminar de filtrar as imagens

# Por fim podemos treinar a rede

In [8]:
# Teste do single shot
f_obj = ImageFilter(verbose = True, single = True)
plt.imshow(f_obj.img)
plt.show()

Insira o nome do filtro 0:
> eq_histogram
eq_histogram
Insira o nome do filtro 1:
> end
end
wanted = ['eq_histogram']
Insira agora o path para a imagem do teste: 
> images/0004/0004_012.bmp
Aplicando Filtro eq_histogram...


NotImplementedError: Class `ImageFilter` does not implement `eq_histogram`

In [ ]:
bw_img = cv2.cvtColor(cv2.imread("images/0042/0042_003.bmp"), cv2.COLOR_BGR2GRAY)
a = ImageFilter(bw_img)
plt.imshow(bw_img, cmap='gray')
plt.show()
a.aplica_filtros()
plt.imshow(a.out, cmap='gray')
plt.show()